In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
import sys
import os
root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if root_path not in sys.path:
    sys.path.append(root_path)
from fileDir import getDataDir
from modules.v1.catboostModel import trainCatboost, testCatboost

In [4]:
VERSION = 3
TRAIN_PATH = getDataDir("train")
TEST_PATH = getDataDir("test")
CLEANED_TRAIN_PATH = getDataDir("train", VERSION)
CLEANED_TEST_PATH = getDataDir("test", VERSION)

## Clean Datasets

In [7]:
cat_features = [
    "Area",
    "Province",
    "Shop Name",
    "gender",
    "date_of_birth",
    "marital_status",
    "tel_category",
    "type_of_residence",
    "c_business_type",
    "c_position",
    "c_occupation",
    "c_employment_status",
    "c_salary_payment_methods",
    "media",
    "place_for_sending_information",
    "r_propose",
    "r_generalcode1",
    "r_generalcode2",
    "r_generalcode3",
    "r_generalcode4",
    "r_generalcode5",
    "apply"
]

In [8]:
def cleanDataset(df, cleanPath):
    # Drop unnecessary columns (keep ID)
    drop_cols = ["ID", "pms_i_ymd","postal_code", "c_postal_code"]
    df = df.drop(columns=drop_cols, errors='ignore')

    # Combine year/month features safely
    if {'living_period_month','living_period_year'}.issubset(df.columns):
        df['living_period_year'] = (df['living_period_month'].fillna(0) + df['living_period_year'].fillna(0) * 12)/12
        df.drop(columns=['living_period_month'], inplace=True)

    if {'c_number_of_working_month','c_number_of_working_year'}.issubset(df.columns):
        df['c_number_of_working_month'] = df['c_number_of_working_month'].fillna(0) + df['c_number_of_working_year'].fillna(0) * 12
        df.drop(columns=['c_number_of_working_year'], inplace=True)

    # Convert birth date to age
    if {'APP_date', 'date_of_birth'}.issubset(df.columns):
        df['APP_date'] = pd.to_datetime(df['APP_date'], errors='coerce')
        df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')
        df['age'] = (df['APP_date'] - df['date_of_birth']).dt.days // 365
        df.drop(columns=['date_of_birth'], inplace=True)

    if {'r_allloan_amount','r_expected_credit_limit'}.issubset(df.columns):
        df["debt_ratio"] = df["r_allloan_amount"] / (df["r_expected_credit_limit"].replace(0, np.nan))
        df["debt_ratio"].fillna(0, inplace=True)

    # Fill categorical columns
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df.replace(["NaN", "nan", "None", ""], np.nan, inplace=True)
    for c in cat_features:
        if c in df.columns:
            df[c] = df[c].fillna('Unknown')

    # Ensure numeric columns are numeric
    num_cols = [col for col in df.columns if col not in cat_features]

    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    num_cols = [c for c in num_cols if c in df.columns]
    df[num_cols] = df[num_cols].fillna(df[num_cols].median())

    print(df.head(3))
    print(df['c_monthly_salary'].min(), df['c_monthly_salary'].max())
    df.to_csv(cleanPath, index=False)
    return df

In [9]:
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)
ids = test_df["ID"].copy()

train_df = cleanDataset(train_df, CLEANED_TRAIN_PATH)
test_df = cleanDataset(test_df, CLEANED_TEST_PATH)

C:\Users\Toa\AppData\Local\Temp\ipykernel_348\504481239.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["debt_ratio"].fillna(0, inplace=True)
C:\Users\Toa\AppData\Local\Temp\ipykernel_348\504481239.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


                          Area Province      Shop Name gender  \
0  Bangkok Metropolitan Region  Bangkok  Silom Complex     F2   
1  Bangkok Metropolitan Region  Bangkok  Silom Complex      M   
2  Bangkok Metropolitan Region  Bangkok  Silom Complex     F2   

   date_of_birth_week date_of_birth  marital_status  number_of_children  \
0                   6     5/18/1971               1                   2   
1                   4      1/5/1983               1                   0   
2                   5    11/19/1992               1                   0   

   tel_category  number_of_resident  ...  r_additional_income  \
0             3                   2  ...                  0.0   
1             3                   3  ...               5000.0   
2             3                   6  ...                  0.0   

   r_spouse_income  r_generalcode1  r_generalcode2  r_generalcode3  \
0              0.0         Unknown         Unknown             2.0   
1              0.0         Unknown   

C:\Users\Toa\AppData\Local\Temp\ipykernel_348\504481239.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["debt_ratio"].fillna(0, inplace=True)
C:\Users\Toa\AppData\Local\Temp\ipykernel_348\504481239.py:27: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


## Testing

In [7]:
threshold = trainCatboost(VERSION, train_df, cat_features)

[I 2025-10-28 21:58:37,251] A new study created in memory with name: no-name-bbce17e5-4cf7-4cb9-b860-6a72861ba34a


GPU detected -> using CatBoost GPU training.
Running hyperparameter tuning with Optuna...


  0%|          | 0/40 [00:00<?, ?it/s]

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


[I 2025-10-28 22:00:08,450] Trial 0 finished with value: 0.6345653499013337 and parameters: {'iterations': 550, 'depth': 8, 'learning_rate': 0.055817288830274696, 'l2_leaf_reg': 8.60211426411632, 'random_strength': 0.15324539525745628, 'bagging_temperature': 0.45220376340945134, 'grow_policy': 'SymmetricTree', 'leaf_estimation_iterations': 4, 'leaf_estimation_method': 'Gradient', 'one_hot_max_size': 4, 'max_ctr_complexity': 2, 'auto_class_weights': None, 'od_type': 'IncToDec', 'od_wait': 57, 'simple_ctr': 'FeatureFreq'}. Best is trial 0 with value: 0.6345653499013337.


...

[I 2025-10-28 23:11:46,279] Trial 39 finished with value: 0.6323273550146188 and parameters: {'iterations': 638, 'depth': 8, 'learning_rate': 0.017869833577608758, 'l2_leaf_reg': 1.4467989109859312, 'random_strength': 2.9770658848659513, 'bagging_temperature': 0.5040870443408507, 'grow_policy': 'SymmetricTree', 'leaf_estimation_iterations': 4, 'leaf_estimation_method': 'Newton', 'one_hot_max_size': 7, 'max_ctr_complexity': 2, 'auto_class_weights': 'Balanced', 'od_type': 'IncToDec', 'od_wait': 56, 'simple_ctr': 'FloatTargetMeanValue'}. Best is trial 24 with value: 0.6372679118933181.

Best Parameters Found:
{'iterations': 1374, 'depth': 8, 'learning_rate': 0.02560971165859926, 'l2_leaf_reg': 0.06477552107369805, 'random_strength': 0.8597240840018328, 'bagging_temperature': 0.3643435728144905, 'grow_policy': 'SymmetricTree', 'leaf_estimation_iterations': 4, 'leaf_estimation_method': 'Gradient', 'one_hot_max_size': 9, 'max_ctr_complexity': 2, 'auto_class_weights': None, 'od_type': 'IncToD

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Change of simpleCtr will not affect combinations ctrs.
Change of simpleCtr will not affect combinations ctrs.
Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6042821	best: 0.6042821 (0)	total: 60.1ms	remaining: 1m 22s
200:	test: 0.6319764	best: 0.6319764 (200)	total: 12.7s	remaining: 1m 13s
400:	test: 0.6347045	best: 0.6347045 (400)	total: 22.5s	remaining: 54.6s
600:	test: 0.6354169	best: 0.6355478 (575)	total: 32.3s	remaining: 41.5s
800:	test: 0.6356610	best: 0.6359806 (780)	total: 42.3s	remaining: 30.2s
1000:	test: 0.6368963	best: 0.6368963 (1000)	total: 52.3s	remaining: 19.5s
1200:	test: 0.6369961	best: 0.6372697 (1160)	total: 1m 2s	remaining: 8.99s
1373:	test: 0.6368786	best: 0.6372697 (1160)	total: 1m 10s	remaining: 0us
bestTest = 0.6372697353
bestIteration = 1160
Shrink model to first 1161 iterations.

Optimal Threshold: 0.481

Confusion Matrix:
 [[3000 2663]
 [ 271  571]]

Classification Report:
               precision    recall  f1-score   support

           0       0.92      0.53      0.67      5663
           1       0.18      0.68      0.28       842

    accuracy                           0.55      6505
   macro avg

In [ ]:
testCatboost(VERSION, test_df, ids, threshold, cat_features)

KeyError: "['ID', 'postal_code', 'age'] not in index"